In [15]:
import os

txtFilesDir = "pages/"
txtFiles = os.listdir(txtFilesDir)

files = []

for file in txtFiles:
    if ".txt" in file:
        files.append(txtFilesDir + file)

files

['pages/HIS-2019-fall.txt',
 'pages/GEG-2019-fall.txt',
 'pages/POL-2020-winter.txt',
 'pages/POL-2019-fall.txt',
 'pages/HIS-2020-winter.txt',
 'pages/GEG-2020-winter.txt']

In [24]:
def extract_file_details(file):
    file_components = file.split(txtFilesDir)[1].split('.txt')[0].split('-')
    
    return {
        'discipline': file_components[0],
        'year': int(file_components[1]),
        'term': file_components[2],
    }

extract_file_details('pages/HIS-2019-fall.txt')

{'discipline': 'HIS', 'term': 'fall', 'year': 2019}

In [26]:
from parser2 import TermSubjectParser

p = TermSubjectParser('HIS', 2019, 'fall')

0 courses removed due to badly formed data.


In [27]:
p.get_courses()

admin.duration  admin.isOpen admin.type  \
course   section                                           
HIS 1101 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 1110 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 1111 B00          FullSess.          True        LEC   
         B00          FullSess.          True        LEC   
         WB00         FullSess.          True        LEC   
HIS 1500 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 1501 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 1520 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2100 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
         B00          FullSess.          True        LEC   
         B00          FullSess.          True        LEC   
HIS 2101 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2103 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2105 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2129 B00          FullSess.          True        LEC   
HIS 2152 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2160 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 2171 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
...                         ...           ...        ...   
HIS 3301 B00          FullSess.          True        LEC   
         B00          FullSess.          True        LEC   
         C00          FullSess.          True        LEC   
HIS 3315 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 3355 A00          FullSess.          True        LEC   
HIS 3397 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 3550 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 3728 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 3765 A00          FullSess.          True        LEC   
HIS 4100 A00          FullSess.          True        SEM   
         B00          FullSess.          True        SEM   
HIS 4135 B00          FullSess.          True        SEM   
         C00          FullSess.          True        SEM   
HIS 4150 A00          FullSess.          True        SEM   
         A00          FullSess.          True        SEM   
HIS 4192 A00          FullSess.          True        SEM   
HIS 4301 A00          FullSess.          True        LEC   
HIS 4361 A00          FullSess.          True        LEC   
         A00          FullSess.          True        LEC   
HIS 4362 A00          FullSess.          True        LEC   
HIS 4380 A00          FullSess.          True        SEM   
HIS 4397 A00          FullSess.          True        SEM   
HIS 4535 A00          FullSess.          True        SEM   
HIS 4551 A00          FullSess.          True        SEM   
HIS 4762 A00          FullSess.          True        LEC   
HIS 4770 A00          FullSess.          True        SEM   

                                    prof weekday time.start time.end  \
course   section                                                       
HIS 1101 A00      Damien-Claude Bélanger      We      13:00    14:20   
         A00      Da

In [14]:
p.courses.query('course == ["HIS 1110", "HIS 3120", "HIS 3124", "HIS 3150", "HIS 4100", "HIS 4192", "HIS 4380", "HIS 4397"]')

admin.duration  admin.isOpen admin.type               prof  \
course   section                                                              
HIS 1110 A00          FullSess.          True        LEC        Eric Allina   
         A00          FullSess.          True        LEC  Meredith Terretta   
HIS 3120 A00          FullSess.          True        LEC              Staff   
HIS 3124 A00          FullSess.          True        LEC    Richard Connors   
         A00          FullSess.          True        LEC    Richard Connors   
HIS 3150 A00          FullSess.          True        LEC     Heather Murray   
         A00          FullSess.          True        LEC     Heather Murray   
HIS 4100 A00          FullSess.          True        SEM        Eric Allina   
         B00          FullSess.          True        SEM     Heather Murray   
HIS 4192 A00          FullSess.          True        SEM              Staff   
HIS 4380 A00          FullSess.          True        SEM   M. Mark Stolarik   
HIS 4397 A00          FullSess.          True        SEM    Richard Connors   

                 weekday time.start time.end                     location  \
course   section                                                            
HIS 1110 A00          Mo      11:30    12:50     125 University (MNT) 202   
         A00          Th      13:00    14:20     125 University (MNT) 202   
HIS 3120 A00          Th      08:30    11:20  129 Louis Pasteur (LPR) 285   
HIS 3124 A00          Mo      10:00    11:20      60 University (SMD) 227   
         A00          We      08:30    09:50      65 University (MRT) 251   
HIS 3150 A00          Tu      13:00    14:20      65 University (MRT) 252   
         A00          Th      11:30    12:50      65 University (MRT) 251   
HIS 4100 A00          Tu      08:30    11:20    120 University (FSS) 9003   
         B00          Th      14:30    17:20        55 Laurier (DMS) 8161   
HIS 4192 A00          We      08:30    11:20   120 University (FSS) 11003   
HIS 4380 A00          Th      19:00    21:50    120 University (FSS) 4013   
HIS 4397 A00          We      14:30    17:20        55 Laurier (DMS) 9143   

                  date.start    date.end  credits  year language  \
course   section                                                   
HIS 1110 A00      2019-09-04  2019-12-03        3     1  English   
         A00      2019-09-04  2019-12-03        3     1  English   
HIS 3120 A00      2019-09-04  2019-12-03        3     3  English   
HIS 3124 A00      2019-09-04  2019-12-03        3     3  English   
         A00      2019-09-04  2019-12-03        3     3  English   
HIS 3150 A00      2019-09-04  2019-12-03        3     3  English   
         A00      2019-09-04  2019-12-03        3     3  English   
HIS 4100 A00      2019-09-04  2019-12-03        3     4  English   
         B00      2019-09-04  2019-12-03        3     4  English   
HIS 4192 A00      2019-09-04  2019-12-03        3     4  English   
HIS 4380 A00      2019-09-04  2019-12-03        3     4  English   
HIS 4397 A00      2019-09-04  2019-12-03        3     4  English   

                                                              title  \
course   section                                                      
HIS 1110 A00                         Introduction to Global History   
         A00                         Introduction to Global History   
HIS 3120 A00      Selected Topics in the History of Canada's Reg...   
HIS 3124 A00                              Britain from 1485 to 1800   
         A00                              Britain from 1485 to 1800   
HIS 3150 A00                    Selected Topics in American History   
         A00                    Selected Topics in American History   
HIS 4100 A00                      Seminar in History Across Borders   
         B00                      Seminar in History Across Borders   
HIS 4192 A00      Seminar in the History of the Middle East and ...   
HIS 4380 A00              